In [1]:
suppressPackageStartupMessages(suppressWarnings(library(ggplot2))) # plotting
suppressPackageStartupMessages(suppressWarnings(library(dplyr))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(patchwork))) # figure composition
suppressPackageStartupMessages(suppressWarnings(library(cowplot))) # figure composition
suppressPackageStartupMessages(suppressWarnings(library(RcppTOML))) # parsing config file
suppressPackageStartupMessages(suppressWarnings(library(lattice))) # heatmap
suppressPackageStartupMessages(suppressWarnings(library(RColorBrewer))) # heatmap
suppressPackageStartupMessages(suppressWarnings(library(ComplexHeatmap))) # heatmap
suppressPackageStartupMessages(suppressWarnings(library(ggplotify))) # grob
suppressPackageStartupMessages(suppressWarnings(library(viridis))) # color
suppressPackageStartupMessages(suppressWarnings(library(platetools))) # make plate plot
suppressPackageStartupMessages(suppressWarnings(library(circlize))) 
suppressPackageStartupMessages(suppressWarnings(library(reshape2))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(stringr))) # string manipulation
suppressPackageStartupMessages(suppressWarnings(library(purrr))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(VennDiagram))) # venn diagram
suppressPackageStartupMessages(suppressWarnings(library(tidyverse))) # data manipulation
suppressPackageStartupMessages(suppressWarnings(library(ggvenn))) # venn diagram
suppressPackageStartupMessages(suppressWarnings(library(grid))) # grid
suppressPackageStartupMessages(suppressWarnings(library(Polychrome)))

source("../../utils/figure_themes.r")


In [2]:
cell_type <- "PBMC"


In [3]:
# set up directories
figure_dir <- file.path("..","figures")
if(!dir.exists(figure_dir)){
    dir.create(figure_dir)
}

## UMAP of morphological features

In [4]:
cell_umap_path <- file.path(paste0(
    "../","../","../","1.Exploratory_Data_Analysis/results/",cell_type,"_umap_values_morphology_sample_100.parquet"
))

cell_umap <- arrow::read_parquet(cell_umap_path)

In [5]:

# Load data
data_path_cytokine_values <- file.path("../../../2.Nomic_nELISA_Analysis/Data/clean/Plate2/nELISA_plate_430420_PBMC_clean.parquet")
cytokine_values <- arrow::read_parquet(data_path_cytokine_values)
# read in the ground truth data
data_path_ground_truth <- file.path("../../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml")
ground_truth <- parseTOML(data_path_ground_truth)
# make a a list of the treatments that are in the ground truth data
apoptosis_ground_truth_list <- c(ground_truth$Apoptosis$apoptosis_groups_list)
pyroptosis_ground_truth_list <- c(ground_truth$Pyroptosis$pyroptosis_groups_list)
control_ground_truth_list <- c(ground_truth$Healthy$healthy_groups_list)

pyroptosis_ground_truth_list
control_ground_truth_list

[1] "Flagellin_1.000_ug_per_ml_DMSO_0.025_%"                     
 [2] "Flagellin_1.000_ug_per_ml_DMSO_0.0_%"                       
 [3] "Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM"              
 [4] "LPS_0.010_ug_per_ml_DMSO_0.025_%"                           
 [5] "LPS_0.100_ug_per_ml_DMSO_0.025_%"                           
 [6] "LPS_1.000_ug_per_ml_DMSO_0.025_%"                           
 [7] "LPS_10.000_ug_per_ml_DMSO_0.025_%"                          
 [8] "LPS_10.000_ug_per_ml_Disulfiram_0.100_uM"                   
 [9] "LPS_10.000_ug_per_ml_Disulfiram_1.000_uM"                   
[10] "LPS_10.000_ug_per_ml_Disulfiram_2.500_uM"                   
[11] "LPS_100.000_ug_per_ml_DMSO_0.025_%"                         
[12] "LPS_Nigericin_1.000_ug_per_ml_1.000_uM_DMSO_0.025_%"        
[13] "LPS_Nigericin_1.000_ug_per_ml_10.000_uM_DMSO_0.025_%"       
[14] "LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Disulfiram_1.000_uM"
[15] "LPS_Nigericin_1.000_ug_per_ml_3.000_uM_DMSO_0.025_%"        
[16] "LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_0.025_%"      
[17] "LPS_Nigericin_100.000_ug_per_ml_10.000_uM_DMSO_0.025_%"     
[18] "LPS_Nigericin_100.000_ug_per_ml_3.000_uM_DMSO_0.025_%"

[1] "DMSO_0.100_%_DMSO_0.025_%"                                   
 [2] "Flagellin_0.100_ug_per_ml_DMSO_0.025_%"                      
 [3] "DMSO_0.100_%_DMSO_1.000_%"                                   
 [4] "DMSO_0.100_%_Z-VAD-FMK_100.000_uM"                           
 [5] "DMSO_0.100_%_Z-VAD-FMK_30.000_uM"                            
 [6] "Disulfiram_0.100_uM_DMSO_0.025_%"                            
 [7] "Disulfiram_1.000_uM_DMSO_0.025_%"                            
 [8] "Disulfiram_2.500_uM_DMSO_0.025_%"                            
 [9] "H2O2_100.000_nM_DMSO_0.025_%"                                
[10] "H2O2_100.000_uM_DMSO_0.025_%"                                
[11] "H2O2_100.000_uM_Disulfiram_1.000_uM"                         
[12] "H2O2_100.000_uM_Z-VAD-FMK_100.000_uM"                        
[13] "LPS_10.000_ug_per_ml_Z-VAD-FMK_100.000_uM"                   
[14] "LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-FMK_100.000_uM"
[15] "Topotecan_10.000_nM_DMSO_0.025_%"                            
[16] "Topotecan_20.000_nM_DMSO_0.025_%"                            
[17] "Topotecan_5.000_nM_DMSO_0.025_%"                             
[18] "media_ctr_0.0_0_Media_0.0_0"                                 
[19] "media_ctr_0.0_0_Media_ctr_0.0_0"

In [6]:


# replace Flagellin_0.100_ug_per_ml_DMSO_0.0_% with Flagellin_0.100_ug_per_ml_DMSO_0.025_%

cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose <- ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose == "Flagellin_0.100_ug_per_ml_DMSO_0.000_%", "Flagellin_0.100_ug_per_ml_DMSO_0.025_%", cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)
cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose <- ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose == "media_ctr_0.0_0_Media_0_0", "media_ctr_0.0_0_Media_ctr_0.0_0", cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)
cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose <- ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose == "Flagellin_1.000_ug_per_ml_DMSO_0.000_%", "Flagellin_1.000_ug_per_ml_DMSO_0.0_%", cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)
cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose <- ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose == "Flagellin_1.000_0_Disulfiram_1.000_uM", "Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM", cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)
cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose <- ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose == "Flagellin_1.000_ug_per_ml_DMSO_0.000_%", "Flagellin_1.000_ug_per_ml_DMSO_0.0_%", cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)
cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose <- ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose == "Flagellin_1.000_0_DMSO_0.025_%", "Flagellin_1.000_ug_per_ml_DMSO_0.0_%", cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)

# make a new column that is the treatment group based on the ground truth data
cell_umap$group <- ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose %in% apoptosis_ground_truth_list, "Apoptosis",
                                ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose %in% pyroptosis_ground_truth_list, "Pyroptosis",
                                       ifelse(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose %in% control_ground_truth_list, "Control", "NA")))
# make the group column a factor
cell_umap$group <- factor(cell_umap$group, levels = c("Control","Apoptosis", "Pyroptosis"))

unique(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)


[1] "LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-FMK_100.000_uM"
 [2] "LPS_Nigericin_100.000_ug_per_ml_3.000_uM_DMSO_0.025_%"       
 [3] "LPS_Nigericin_1.000_ug_per_ml_10.000_uM_DMSO_0.025_%"        
 [4] "DMSO_0.100_%_DMSO_0.025_%"                                   
 [5] "LPS_Nigericin_1.000_ug_per_ml_1.000_uM_DMSO_0.025_%"         
 [6] "LPS_0.100_ug_per_ml_DMSO_0.025_%"                            
 [7] "Flagellin_0.100_ug_per_ml_DMSO_0.025_%"                      
 [8] "LPS_10.000_ug_per_ml_DMSO_0.025_%"                           
 [9] "H2O2_100.000_nM_DMSO_0.025_%"                                
[10] "DMSO_0.100_%_DMSO_1.000_%"                                   
[11] "LPS_10.000_ug_per_ml_Disulfiram_1.000_uM"                    
[12] "Thapsigargin_10.000_uM_DMSO_0.025_%"                         
[13] "Disulfiram_0.100_uM_DMSO_0.025_%"                            
[14] "LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Disulfiram_1.000_uM" 
[15] "Thapsigargin_1.000_uM_DMSO_0.025_%"                          
[16] "LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_0.025_%"       
[17] "H2O2_100.000_uM_DMSO_0.025_%"                                
[18] "Flagellin_1.000_ug_per_ml_DMSO_0.0_%"                        
[19] "LPS_Nigericin_1.000_ug_per_ml_3.000_uM_DMSO_0.025_%"         
[20] "DMSO_0.100_%_Z-VAD-FMK_30.000_uM"                            
[21] "media_ctr_0.0_0_Media_ctr_0.0_0"                             
[22] "LPS_100.000_ug_per_ml_DMSO_0.025_%"                          
[23] "LPS_Nigericin_100.000_ug_per_ml_10.000_uM_DMSO_0.025_%"      
[24] "Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM"               
[25] "LPS_10.000_ug_per_ml_Z-VAD-FMK_100.000_uM"                   
[26] "LPS_10.000_ug_per_ml_Disulfiram_0.100_uM"                    
[27] "LPS_1.000_ug_per_ml_DMSO_0.025_%"                            
[28] "Topotecan_20.000_nM_DMSO_0.025_%"                            
[29] "Flagellin_1.000_ug_per_ml_DMSO_0.025_%"                      
[30] "Disulfiram_1.000_uM_DMSO_0.025_%"                            
[31] "Topotecan_5.000_nM_DMSO_0.025_%"                             
[32] "Topotecan_10.000_nM_DMSO_0.025_%"                            
[33] "H2O2_100.000_uM_Z-VAD-FMK_100.000_uM"                        
[34] "LPS_10.000_ug_per_ml_Disulfiram_2.500_uM"                    
[35] "Disulfiram_2.500_uM_DMSO_0.025_%"                            
[36] "H2O2_100.000_uM_Disulfiram_1.000_uM"                         
[37] "LPS_0.010_ug_per_ml_DMSO_0.025_%"                            
[38] "DMSO_0.100_%_Z-VAD-FMK_100.000_uM"

In [8]:
# mutate the names of each treatment

cell_umap <- cell_umap %>%
    mutate(oneb_Metadata_Treatment_Dose_Inhibitor_Dose = case_when(
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='DMSO_0.100_%_DMSO_0.025_%' ~ "DMSO 0.1% - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='DMSO_0.100_%_DMSO_1.000_%' ~ "DMSO 0.1% - DMSO 1.0%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='DMSO_0.100_%_Z-VAD-FMK_100.000_uM' ~ "DMSO 0.1% - Z-VAD-FMK 100.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='DMSO_0.100_%_Z-VAD-FMK_30.000_uM' ~ "DMSO 0.1% - Z-VAD-FMK 30.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Flagellin_1.000_ug_per_ml_DMSO_0.025_%' ~ "Flagellin 1.0 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM' ~ "Flagellin 1.0 ug/ml - Disulfiram 1.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_0.010_ug_per_ml_DMSO_0.025_%' ~ "LPS 0.01 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_0.100_ug_per_ml_DMSO_0.025_%' ~ "LPS 0.1 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Flagellin_0.100_ug_per_ml_DMSO_0.0_%' ~ "Flagellin 0.1 ug/ml - DMSO 0.0%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Flagellin_0.100_ug_per_ml_DMSO_0.025_%' ~ "Flagellin 0.1 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Disulfiram_0.100_uM_DMSO_0.025_%' ~ "Disulfiram 0.1 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_1.000_ug_per_ml_1.000_uM_DMSO_0.025_%' ~ "LPS 1.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_1.000_ug_per_ml_10.000_uM_DMSO_0.025_%' ~ "LPS 1.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Disulfiram_1.000_uM' ~ "LPS 1.0 ug/ml + Nigericin 10.0 uM - Disulfiram 1.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-FMK_100.000_uM' ~ "LPS 1.0 ug/ml + Nigericin 10.0 uM - Z-VAD-FMK 100.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_1.000_ug_per_ml_3.000_uM_DMSO_0.025_%' ~ "LPS 1.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_1.000_ug_per_ml_DMSO_0.025_%' ~ "LPS 1.0 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Flagellin_1.000_ug_per_ml_DMSO_0.0_%' ~ "Flagellin 1.0 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Disulfiram_1.000_uM_DMSO_0.025_%' ~ "Disulfiram 1.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Thapsigargin_1.000_uM_DMSO_0.025_%' ~ "Thapsigargin 1.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Topotecan_10.000_nM_DMSO_0.025_%' ~ "Topotecan 10.0 nM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_10.000_ug_per_ml_DMSO_0.025_%' ~ "LPS 10.0 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_10.000_ug_per_ml_Disulfiram_0.100_uM' ~ "LPS 10.0 ug/ml - Disulfiram 0.1 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_10.000_ug_per_ml_Disulfiram_1.000_uM' ~ "LPS 10.0 ug/ml - Disulfiram 1.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_10.000_ug_per_ml_Disulfiram_2.500_uM' ~ "LPS 10.0 ug/ml - Disulfiram 2.5 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_10.000_ug_per_ml_Z-VAD-FMK_100.000_uM' ~ "LPS 10.0 ug/ml - Z-VAD-FMK 100.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Thapsigargin_10.000_uM_DMSO_0.025_%' ~ "Thapsigargin 10.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='H2O2_100.000_nM_DMSO_0.025_%' ~ "H2O2 100.0 nM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_100.000_ug_per_ml_1.000_uM_DMSO_0.025_%' ~ "LPS 100.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_100.000_ug_per_ml_10.000_uM_DMSO_0.025_%' ~ "LPS 100.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_Nigericin_100.000_ug_per_ml_3.000_uM_DMSO_0.025_%' ~ "LPS 100.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='LPS_100.000_ug_per_ml_DMSO_0.025_%' ~ "LPS 100.0 ug/ml - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='H2O2_100.000_uM_DMSO_0.025_%' ~ "H2O2 100.0 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='H2O2_100.000_uM_Disulfiram_1.000_uM' ~ "H2O2 100.0 uM - Disulfiram 1.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='H2O2_100.000_uM_Z-VAD-FMK_100.000_uM' ~ "H2O2 100.0 uM - Z-VAD-FMK 100.0 uM",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Disulfiram_2.500_uM_DMSO_0.025_%' ~ "Disulfiram 2.5 uM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Topotecan_20.000_nM_DMSO_0.025_%' ~ "Topotecan 20.0 nM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='Topotecan_5.000_nM_DMSO_0.025_%' ~ "Topotecan 5.0 nM - DMSO 0.025%",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='media_ctr_0.0_0_Media_ctr_0.0_0' ~ "Media ctr 0.0 0",
        oneb_Metadata_Treatment_Dose_Inhibitor_Dose =='media_ctr_0.0_0_Media_0.0_0' ~ "Media ctr 0.0 0"
    ))
    # replace Media ctr 0.0 0 with Media
cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose <- gsub("Media ctr 0.0 0", "Media", cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)

unique(cell_umap$oneb_Metadata_Treatment_Dose_Inhibitor_Dose)

[1] "LPS 1.0 ug/ml + Nigericin 10.0 uM - Z-VAD-FMK 100.0 uM"
 [2] "LPS 100.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%"      
 [3] "LPS 1.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%"       
 [4] "DMSO 0.1% - DMSO 0.025%"                               
 [5] "LPS 1.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%"        
 [6] "LPS 0.1 ug/ml - DMSO 0.025%"                           
 [7] "Flagellin 0.1 ug/ml - DMSO 0.025%"                     
 [8] "LPS 10.0 ug/ml - DMSO 0.025%"                          
 [9] "H2O2 100.0 nM - DMSO 0.025%"                           
[10] "DMSO 0.1% - DMSO 1.0%"                                 
[11] "LPS 10.0 ug/ml - Disulfiram 1.0 uM"                    
[12] "Thapsigargin 10.0 uM - DMSO 0.025%"                    
[13] "Disulfiram 0.1 uM - DMSO 0.025%"                       
[14] "LPS 1.0 ug/ml + Nigericin 10.0 uM - Disulfiram 1.0 uM" 
[15] "Thapsigargin 1.0 uM - DMSO 0.025%"                     
[16] "LPS 100.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%"      
[17] "H2O2 100.0 uM - DMSO 0.025%"                           
[18] "Flagellin 1.0 ug/ml - DMSO 0.025%"                     
[19] "LPS 1.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%"        
[20] "DMSO 0.1% - Z-VAD-FMK 30.0 uM"                         
[21] "Media"                                                 
[22] "LPS 100.0 ug/ml - DMSO 0.025%"                         
[23] "LPS 100.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%"     
[24] "Flagellin 1.0 ug/ml - Disulfiram 1.0 uM"               
[25] "LPS 10.0 ug/ml - Z-VAD-FMK 100.0 uM"                   
[26] "LPS 10.0 ug/ml - Disulfiram 0.1 uM"                    
[27] "LPS 1.0 ug/ml - DMSO 0.025%"                           
[28] "Topotecan 20.0 nM - DMSO 0.025%"                       
[29] "Disulfiram 1.0 uM - DMSO 0.025%"                       
[30] "Topotecan 5.0 nM - DMSO 0.025%"                        
[31] "Topotecan 10.0 nM - DMSO 0.025%"                       
[32] "H2O2 100.0 uM - Z-VAD-FMK 100.0 uM"                    
[33] "LPS 10.0 ug/ml - Disulfiram 2.5 uM"                    
[34] "Disulfiram 2.5 uM - DMSO 0.025%"                       
[35] "H2O2 100.0 uM - Disulfiram 1.0 uM"                     
[36] "LPS 0.01 ug/ml - DMSO 0.025%"                          
[37] "DMSO 0.1% - Z-VAD-FMK 100.0 uM"

In [10]:
head(cell_umap)
# split the oneb_Metadata_Treatment_Dose_Inhibitor_Dose into two columns by the " - " delimiter
cell_umap <- cell_umap %>%
    separate(oneb_Metadata_Treatment_Dose_Inhibitor_Dose, c("inducer", "inhibitor"), sep = " - ", remove = FALSE)
head(cell_umap)

Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,⋯,Nuclei_Texture_SumEntropy_CorrPM_3_01_256,Nuclei_Texture_SumVariance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrER_3_00_256,Nuclei_Texture_Variance_CorrGasdermin_3_00_256,umap_1,umap_2,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,__index_level_0__,__index_level_1__,group
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>,<fct>
-0.06877138,-0.1413193089,-0.4610874,-0.2981762,-1.4737896,-0.42951796,-0.3870007,0.5810070,-0.5510293,0.07787682,⋯,-0.9103790,-0.1512759962,-0.1915608,-0.1779862,-1.549306,10.011412,LPS 1.0 ug/ml + Nigericin 10.0 uM - Z-VAD-FMK 100.0 uM,LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-FMK_100.000_uM,5379859,Control
-1.61733620,1.6893310003,-1.4184128,-1.9574187,1.0815975,0.51658710,-0.7941781,0.7493335,0.3621578,-0.97664790,⋯,-2.0178383,-0.1690371729,-0.1986381,-0.1948380,-0.628924,10.268293,LPS 100.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%,LPS_Nigericin_100.000_ug_per_ml_3.000_uM_DMSO_0.025_%,742943,Pyroptosis
0.04458081,-0.2291663309,-0.3167262,-0.8992577,-1.1030220,-0.68043480,-0.6386509,-0.9854598,0.3214117,-0.42885935,⋯,-1.8030197,-0.1676858066,-0.1946570,-0.1931246,-2.095922,5.390349,LPS 1.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%,LPS_Nigericin_1.000_ug_per_ml_10.000_uM_DMSO_0.025_%,4135253,Pyroptosis
0.57605261,-0.5914704046,-0.0652136,0.8567671,-1.4871873,-0.60385952,-0.5873071,1.0431710,-0.8796077,-0.36710875,⋯,-0.8374835,-0.1575222483,-0.2050011,-0.1906528,-4.475632,9.272793,DMSO 0.1% - DMSO 0.025%,DMSO_0.100_%_DMSO_0.025_%,2756279,Control
-0.24101402,0.0005652865,-0.3611522,-0.2796615,0.9809175,0.55256609,0.8588446,-0.1983269,0.6690683,2.20892618,⋯,0.9349406,0.0007495165,0.1714802,0.1549115,1.114986,4.273778,LPS 1.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%,LPS_Nigericin_1.000_ug_per_ml_1.000_uM_DMSO_0.025_%,3722431,Pyroptosis
-0.60057746,0.3344381520,-0.8929653,-0.9229199,0.5022947,0.08376447,-0.6235339,1.2532222,-0.4123834,-0.50823133,⋯,-1.5690851,-0.1634222776,-0.1643490,-0.1611734,-3.443907,9.606824,LPS 0.1 ug/ml - DMSO 0.025%,LPS_0.100_ug_per_ml_DMSO_0.025_%,542470,Pyroptosis


Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 200 rows [31, 54, 57,
115, 124, 150, 159, 194, 204, 206, 211, 219, 234, 259, 292, 309, 317, 322, 324,
333, ...].”


Cytoplasm_AreaShape_Compactness,Cytoplasm_AreaShape_FormFactor,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_MinorAxisLength,Cytoplasm_AreaShape_Orientation,Cytoplasm_AreaShape_Zernike_0_0,Cytoplasm_AreaShape_Zernike_1_1,Cytoplasm_AreaShape_Zernike_2_0,Cytoplasm_AreaShape_Zernike_2_2,Cytoplasm_AreaShape_Zernike_3_1,⋯,Nuclei_Texture_Variance_CorrER_3_00_256,Nuclei_Texture_Variance_CorrGasdermin_3_00_256,umap_1,umap_2,oneb_Metadata_Treatment_Dose_Inhibitor_Dose,inducer,inhibitor,__index_level_0__,__index_level_1__,group
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<fct>
-0.06877138,-0.1413193089,-0.4610874,-0.2981762,-1.4737896,-0.42951796,-0.3870007,0.5810070,-0.5510293,0.07787682,⋯,-0.1915608,-0.1779862,-1.549306,10.011412,LPS 1.0 ug/ml + Nigericin 10.0 uM - Z-VAD-FMK 100.0 uM,LPS 1.0 ug/ml + Nigericin 10.0 uM,Z-VAD-FMK 100.0 uM,LPS_Nigericin_1.000_ug_per_ml_10.000_uM_Z-VAD-FMK_100.000_uM,5379859,Control
-1.61733620,1.6893310003,-1.4184128,-1.9574187,1.0815975,0.51658710,-0.7941781,0.7493335,0.3621578,-0.97664790,⋯,-0.1986381,-0.1948380,-0.628924,10.268293,LPS 100.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%,LPS 100.0 ug/ml + Nigericin 3.0 uM,DMSO 0.025%,LPS_Nigericin_100.000_ug_per_ml_3.000_uM_DMSO_0.025_%,742943,Pyroptosis
0.04458081,-0.2291663309,-0.3167262,-0.8992577,-1.1030220,-0.68043480,-0.6386509,-0.9854598,0.3214117,-0.42885935,⋯,-0.1946570,-0.1931246,-2.095922,5.390349,LPS 1.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%,LPS 1.0 ug/ml + Nigericin 10.0 uM,DMSO 0.025%,LPS_Nigericin_1.000_ug_per_ml_10.000_uM_DMSO_0.025_%,4135253,Pyroptosis
0.57605261,-0.5914704046,-0.0652136,0.8567671,-1.4871873,-0.60385952,-0.5873071,1.0431710,-0.8796077,-0.36710875,⋯,-0.2050011,-0.1906528,-4.475632,9.272793,DMSO 0.1% - DMSO 0.025%,DMSO 0.1%,DMSO 0.025%,DMSO_0.100_%_DMSO_0.025_%,2756279,Control
-0.24101402,0.0005652865,-0.3611522,-0.2796615,0.9809175,0.55256609,0.8588446,-0.1983269,0.6690683,2.20892618,⋯,0.1714802,0.1549115,1.114986,4.273778,LPS 1.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%,LPS 1.0 ug/ml + Nigericin 1.0 uM,DMSO 0.025%,LPS_Nigericin_1.000_ug_per_ml_1.000_uM_DMSO_0.025_%,3722431,Pyroptosis
-0.60057746,0.3344381520,-0.8929653,-0.9229199,0.5022947,0.08376447,-0.6235339,1.2532222,-0.4123834,-0.50823133,⋯,-0.1643490,-0.1611734,-3.443907,9.606824,LPS 0.1 ug/ml - DMSO 0.025%,LPS 0.1 ug/ml,DMSO 0.025%,LPS_0.100_ug_per_ml_DMSO_0.025_%,542470,Pyroptosis


In [14]:
unique(cell_umap$inducer)
unique(cell_umap$inhibitor)
# replace the inhibitor NA with Media
cell_umap$inhibitor <- ifelse(is.na(cell_umap$inhibitor), "Media", cell_umap$inhibitor)
unique(cell_umap$inhibitor)

[1] LPS 1.0 ug/ml + Nigericin 10.0 uM   LPS 100.0 ug/ml + Nigericin 3.0 uM 
 [3] DMSO 0.1%                           LPS 1.0 ug/ml + Nigericin 1.0 uM   
 [5] LPS 0.1 ug/ml                       Flagellin 0.1 ug/ml                
 [7] LPS 10.0 ug/ml                      H2O2 100.0 nM                      
 [9] Thapsigargin 10.0 uM                Disulfiram 0.1 uM                  
[11] Thapsigargin 1.0 uM                 LPS 100.0 ug/ml + Nigericin 1.0 uM 
[13] H2O2 100.0 uM                       Flagellin 1.0 ug/ml                
[15] LPS 1.0 ug/ml + Nigericin 3.0 uM    Media                              
[17] LPS 100.0 ug/ml                     LPS 100.0 ug/ml + Nigericin 10.0 uM
[19] LPS 1.0 ug/ml                       Topotecan 20.0 nM                  
[21] Disulfiram 1.0 uM                   Topotecan 5.0 nM                   
[23] Topotecan 10.0 nM                   Disulfiram 2.5 uM                  
[25] LPS 0.01 ug/ml                     
25 Levels: Media DMSO 0.1% Flagellin 0.1 ug/ml ... Topotecan 20.0 nM

[1] "Z-VAD-FMK 100.0 uM" "DMSO 0.025%"        "DMSO 1.0%"         
[4] "Disulfiram 1.0 uM"  "Z-VAD-FMK 30.0 uM"  NA                  
[7] "Disulfiram 0.1 uM"  "Disulfiram 2.5 uM"

[1] "Z-VAD-FMK 100.0 uM" "DMSO 0.025%"        "DMSO 1.0%"         
[4] "Disulfiram 1.0 uM"  "Z-VAD-FMK 30.0 uM"  "Media"             
[7] "Disulfiram 0.1 uM"  "Disulfiram 2.5 uM"

In [15]:
# make the group_treatment column a factor
cell_umap$inducer <- factor(
    cell_umap$inducer, 
    levels = c(
        'Media',
        'DMSO 0.1%',

        'Flagellin 0.1 ug/ml',
        'Flagellin 1.0 ug/ml',

        'LPS 0.01 ug/ml',
        'LPS 0.1 ug/ml',
        'LPS 1.0 ug/ml',
        'LPS 10.0 ug/ml',
        'LPS 100.0 ug/ml',

        'LPS 1.0 ug/ml + Nigericin 1.0 uM',
        'LPS 1.0 ug/ml + Nigericin 3.0 uM',
        'LPS 1.0 ug/ml + Nigericin 10.0 uM',
        
        'LPS 100.0 ug/ml + Nigericin 1.0 uM',
        'LPS 100.0 ug/ml + Nigericin 3.0 uM',
        'LPS 100.0 ug/ml + Nigericin 10.0 uM',
        
        'H2O2 100.0 nM',
        'H2O2 100.0 uM',
        
        'Disulfiram 0.1 uM',
        'Disulfiram 1.0 uM',
        'Disulfiram 2.5 uM',

        'Thapsigargin 1.0 uM',
        'Thapsigargin 10.0 uM',

        'Topotecan 5.0 nM',
        'Topotecan 10.0 nM',
        'Topotecan 20.0 nM'
    )
)

# make the group_treatment column a factor
cell_umap$inhibitor <- factor(
    cell_umap$inhibitor, 
    levels = c(
        'Media',
        'DMSO 0.025%',
        'DMSO 1.0%',
        
        'Disulfiram 0.1 uM',
        'Disulfiram 1.0 uM',
        'Disulfiram 2.5 uM',
        
        'Z-VAD-FMK 30.0 uM',
        'Z-VAD-FMK 100.0 uM'
    )
)

            
            

In [17]:
colors <- c(
    'Media' = colorgrad1[1],
    'DMSO 0.1%' = colorgrad1[2],

    'Disulfiram 0.1 uM' = colorgrad4[3],
    'Disulfiram 1.0 uM' = colorgrad4[4],
    'Disulfiram 2.5 uM' = colorgrad4[5],

    'Flagellin 0.1 ug/ml'= colorgrad5[1],
    'Flagellin 1.0 ug/ml' = colorgrad5[2],

    'LPS 0.01 ug/ml' = colorgrad2[1],
    'LPS 0.1 ug/ml' = colorgrad2[2],
    'LPS 1.0 ug/ml' = colorgrad2[3],
    'LPS 10.0 ug/ml - DMSO 0.025%' = colorgrad2[4],
    'LPS 100.0 ug/ml' = colorgrad2[5],
    'LPS 1.0 ug/ml + Nigericin 1.0 uMs' = colorgrad2[6],
    'LPS 1.0 ug/ml + Nigericin 3.0 uM' = colorgrad2[7],
    'LPS 1.0 ug/ml + Nigericin 10.0 uM' = colorgrad2[8],
    'LPS 100.0 ug/ml + Nigericin 1.0 uM' = colorgrad2[9],
    'LPS 100.0 ug/ml + Nigericin 3.0 uM' = colorgrad2[10],
    'LPS 100.0 ug/ml + Nigericin 10.0 uM' = colorgrad2[11],

    'H2O2 100.0 nM'  = colorgrad7[1],
    'H2O2 100.0 uM' = colorgrad7[2],

    'Thapsigargin 1.0 uM' = colorgrad8[1],
    'Thapsigargin 10.0 uM' = colorgrad8[2],

    'Topotecan 5.0 nM' = colorgrad9[1],
    'Topotecan 10.0 nM' = colorgrad9[2],
    'Topotecan 20.0 nM' = colorgrad9[3]

)

ERROR: Error in eval(expr, envir, enclos): object 'colorgrad1' not found


In [ ]:
# define the shaps of the points
shape_list <- c(
    19, # Media
    19, # DMSO 0.025%
    8, # DMSO 1.0%
    18, # Disulfiram 0.1 uM
    18, # Disulfiram 1.0 uM
    18, # Disulfiram 2.5 uM
    17, # Z-VAD-FMK 30.0 uM
    17 # Z-VAD-FMK 100.0 uM
)

In [ ]:

shapes <- c(
    'Media, Control' = shape_list[1],
    'DMSO 0.1% - DMSO 0.025%, Control' = shape_list[2],
    'DMSO 0.1% - DMSO 1.0%, Control' = shape_list[3],
    'DMSO 0.1% - Z-VAD-FMK 30.0 uM, Control'= shape_list[7],
    'DMSO 0.1% - Z-VAD-FMK 100.0 uM, Control'= shape_list[8],


    'Disulfiram 0.1 uM - DMSO 0.025%, Control' = shape_list[2],

    'Disulfiram 1.0 uM - DMSO 0.025%, Control' = shape_list[2],

    'Disulfiram 2.5 uM - DMSO 0.025%, Control' = shape_list[2],


    'Flagellin 0.1 ug/ml - DMSO 0.025%, Control'= shape_list[2],

    'Flagellin 1.0 ug/ml - DMSO 0.025%, Pyroptosis' = shape_list[2],
    'Flagellin 1.0 ug/ml - Disulfiram 1.0 uM, Pyroptosis' = shape_list[5],

    'LPS 0.01 ug/ml - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 0.1 ug/ml - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 1.0 ug/ml - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 1.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 1.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 1.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%, Pyroptosis' = shape_list[2],
    'LPS 1.0 ug/ml + Nigericin 10.0 uM - Z-VAD-FMK 100.0 uM, Control' = shape_list[8],
    'LPS 1.0 ug/ml + Nigericin 10.0 uM - Disulfiram 1.0 uM, Pyroptosis' = shape_list[5],

    'LPS 10.0 ug/ml - DMSO 0.025%, Pyroptosis' = shape_list[2],
    'LPS 10.0 ug/ml - Z-VAD-FMK 100.0 uM, Control' = shape_list[8],
    'LPS 10.0 ug/ml - Disulfiram 0.1 uM, Pyroptosis' = shape_list[4],
    'LPS 10.0 ug/ml - Disulfiram 1.0 uM, Pyroptosis' = shape_list[5],
    'LPS 10.0 ug/ml - Disulfiram 2.5 uM, Pyroptosis' = shape_list[6],


    'LPS 100.0 ug/ml - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 100.0 ug/ml + Nigericin 1.0 uM - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 100.0 ug/ml + Nigericin 3.0 uM - DMSO 0.025%, Pyroptosis' = shape_list[2],

    'LPS 100.0 ug/ml + Nigericin 10.0 uM - DMSO 0.025%, Pyroptosis' = shape_list[2],






    'H2O2 100.0 nM - DMSO 0.025%, Control'  = shape_list[2],

    'H2O2 100.0 uM - DMSO 0.025%, Control' = shape_list[2],
    'H2O2 100.0 uM - Disulfiram 1.0 uM, Control' = shape_list[5],
    'H2O2 100.0 uM - Z-VAD-FMK 100.0 uM, Control' = shape_list[8],

    'Thapsigargin 1.0 uM - DMSO 0.025%, Apoptosis' = shape_list[2],

    'Thapsigargin 10.0 uM - DMSO 0.025%, Apoptosis' = shape_list[2],

    'Topotecan 5.0 nM - DMSO 0.025%, Control' = shape_list[2],

    'Topotecan 10.0 nM - DMSO 0.025%, Control' = shape_list[2],

    'Topotecan 20.0 nM - DMSO 0.025%, Control' = shape_list[2]
)


In [ ]:
custom_pallette = createPalette(37,  c("#ff0000", "#00ff00", "#0000ff"))
custom_pallette <- sortByHue(custom_pallette)
custom_pallette <- as.vector(t(matrix(custom_pallette, ncol=4)))


In [ ]:
# set plot size 
width <- 17
height <- 15
options(repr.plot.width = width, repr.plot.height = height)
umap_plot_all <- (
    ggplot(cell_umap, aes(x = umap_1, y = umap_2)) 

    + geom_point(
        aes(
            color = inducer,
            shape = inhibitor
        ), 
        size = 4, 
        alpha = 0.7
    )
    + theme_bw()

    # rename legend title
    + labs(color = "Treatment", hjust = 0.5)
    + figure_theme
        + theme(axis.text.x = element_text(angle = 0, vjust = 0.5, hjust=1))
            + theme(
         legend.text = element_text(size = 16),
        legend.title = element_text(size = 20, hjust = 0.5))
    + scale_color_manual(
        name = "Treatment",
        labels = c(
            
        ),
        values = colors)
    + scale_shape_manual(
        name = "Treatment",
        labels = c(
            
        ),
        values = shapes
    )
    + theme(legend.position = "bottom")
    # set the legend columns to 4
    # change legend alpha 
    + guides(color = guide_legend(ncol =2, override.aes = list(alpha = 1, size = 5)))
    + labs(
        x = "UMAP 1",
        y = "UMAP 2"
    )   
)
umap_plot_all
# save plot
ggsave(
    filename = file.path("..","figures", "umap_plot_all.png"),
    plot = umap_plot_all,
    width = width,
    height = height,
    units = "in",
    dpi = 600
)


In [ ]:
# make a facet plot with all the groups 

In [ ]:
sessionInfo()